In [88]:
from __future__ import division, print_function, absolute_import

import h5py
import os
import pandas as pd
import tflearn
import tensorflow as tf
from tflearn.data_utils import build_hdf5_image_dataset

In [14]:
TRAIN = 'training.csv'
#VAL = 'validation.csv'
TRAIN_TRIM = 'training_trim.csv'

FULL_COLS = ['file_path','face_x','face_y','face_width','face_height','facial_landmarks','expression','valence','arousal']
TRIM_COLS = ['file_path', 'face_x', 'face_y', 'face_width', 'face_height', 'expression']

training = pd.read_csv(os.environ['HOME'] + '/' + TRAIN)
#validation = pd.read_csv(VAL, names=FULL_COLS)

In [15]:
TRAIN_TRIM = 'training_trim.csv'

FULL_COLS = ['file_path','face_x','face_y','face_width','face_height','facial_landmarks','expression','valence','arousal']
TRIM_COLS = ['file_path', 'face_x', 'face_y', 'face_width', 'face_height', 'expression']
training.columns = FULL_COLS
training = training[TRIM_COLS]
training.to_csv(TRAIN_TRIM)

In [16]:
TRAIN_FILE = 'training_trim.csv'
DATASET_FILE = 'train_dataset.txt'
TRAIN_H5 = 'train.h5'
IMG_SIZE = 256
DATA_DIR = os.environ['HOME']

In [23]:
train = pd.read_csv(TRAIN_FILE)
f = open(DATASET_FILE, 'w')

for idx, row in training.iterrows():
    file_path = os.environ['HOME'] + '/' + row['file_path'].split('/')[1]
    if not os.path.isfile(file_path): continue
    # crop image
    # write logic to toss out poorly formatted images
    line_string = '{} {}\n'.format(file_path, row['expression'])
    f.write(line_string)

f.close()

Processed 1000 images
Processed 2000 images
Processed 3000 images
Processed 4000 images
Processed 5000 images
Processed 6000 images
Processed 7000 images
Processed 8000 images
Processed 9000 images
Processed 10000 images
Processed 11000 images
Processed 12000 images
Processed 13000 images
Processed 14000 images
Processed 15000 images
Processed 16000 images
Processed 17000 images
Processed 18000 images
Processed 19000 images
Processed 20000 images
Processed 21000 images
Processed 22000 images
Processed 23000 images
Processed 24000 images
Processed 25000 images
Processed 26000 images
Processed 27000 images
Processed 28000 images
Processed 29000 images
Processed 30000 images
Processed 31000 images
Processed 32000 images
Processed 33000 images
Processed 34000 images
Processed 35000 images
Processed 36000 images
Processed 37000 images


In [63]:
dataset_file = 'train_dataset_trim.txt'
build_hdf5_image_dataset(dataset_file, image_shape=(IMG_SIZE, IMG_SIZE), mode='file', 
                         output_path=TRAIN_H5, normalize=True, categorical_labels=True)

In [64]:
h5f = h5py.File(TRAIN_H5, 'r')
X = h5f['X']
Y = h5f['Y']

In [69]:
# Residual blocks
# 32 layers: n=5, 56 layers: n=9, 110 layers: n=18

In [80]:
# Real-time data preprocessing
img_prep = tflearn.ImagePreprocessing()
img_prep.add_featurewise_zero_center(per_channel=True)


# Real-time data augmentation
img_aug = tflearn.ImageAugmentation()
img_aug.add_random_flip_leftright()
img_aug.add_random_crop([32, 32], padding=4)

In [89]:
n = 5
tf.reset_default_graph()
# Building Residual Network
net = tflearn.input_data(shape=[None, 32, 32, 3],
                         data_preprocessing=img_prep,
                         data_augmentation=img_aug)
net = tflearn.conv_2d(net, 16, 3, regularizer='L2', weight_decay=0.0001)
net = tflearn.resnext_block(net, n, 16, 32)
net = tflearn.resnext_block(net, 1, 32, 32, downsample=True)
net = tflearn.resnext_block(net, n-1, 32, 32)
net = tflearn.resnext_block(net, 1, 64, 32, downsample=True)
net = tflearn.resnext_block(net, n-1, 64, 32)
net = tflearn.batch_normalization(net)
net = tflearn.activation(net, 'relu')
net = tflearn.global_avg_pool(net)

In [90]:
# Training
net = tflearn.fully_connected(net, 11, activation='softmax')
opt = tflearn.Momentum(0.1, lr_decay=0.1, decay_step=32000, staircase=True)
net = tflearn.regression(net, optimizer=opt,
                         loss='categorical_crossentropy')

In [ ]:
model = tflearn.DNN(net, checkpoint_path='model_resnet_affectnet',
                    max_checkpoints=10, tensorboard_verbose=0,
                    clip_gradients=0.)

In [ ]:
model.fit(X, Y, n_epoch=1, validation_set=(X, Y),
          snapshot_epoch=False, snapshot_step=500,
          show_metric=True, batch_size=128, shuffle=True,
          run_id='resnet_affectnet')